## Threat Hunting is More Than Data Retrieval

Until now, we are mostly doing *data retrieval*. Of course, we as threat hunters provide more values than blindly data retrieval:
- patterns to match
- directions to investigate (child process, network traffic of process, etc.)
- suspicious entity identification (grouped by Kestrel variables)

However, reading the data retrieved and thinking what data to retrieve is not enough. We also need:
- data enrichment from other sources like threat intelligence
- visualizations to help us digest data
- pre-programmed detection logic (in white-box or black-box)

Let's `APPLY` any logic (not coded in Kestrel) as a hunt step to perform Turing-complete analysis on a Kestrel variable.

It is easy to wrap a white-box or black-box logic into a Kestrel analytics. Some examples in the [kestrel-analytics repo](https://github.com/opencybersecurityalliance/kestrel-analytics):
- [attribute-plot](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/attributeplot): plot/visualize select attributes of entities.
- [exfiltration-modeling](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/dataexfiltration): infer the likelihood of data exfiltration on input network-traffic.
- [domain-enrichment](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/domainnamelookup): whois lookup and domain information enrichment to network-traffic.
- [log4shell-deobfuscation-and-detection](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/log4shell): log4shell URL de-obfuscation and detection.
- [pin-IP-on-map](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/piniponmap): find geo-location of IP addresses and pin them on a map.
- [SANS-enrich](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/sansipenrich): threat intelligence enrichment with SANS API.
- [scikit-learn-clustering](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/sklearn-cluster): cluster entities using scikit-learn.
- [suspicious-process-scoring](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/suspiciousscoring): compute how suspicious a process is based on domain knowledge like SIGMA.
- [xfe-enrich](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/xfeipenrich): Threat Intelligence enrichment using X-Force Exchange.

In [1]:
# let's get network traffic from suspicious (possible C2) process before we apply analytics

splunkd = GET process FROM stixshifter://bh22-linux-192.168.56.91
          WHERE [process:name = 'splunkd']
          START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'
          
splunkd_activities = FIND process CREATED BY splunkd

traffic = FIND network-traffic CREATED BY splunkd_activities

DISP traffic ATTR src_ref.value, src_port, dst_ref.value, dst_port

src_ref.value 
 src_port 
 dst_ref.value 
 dst_port 
 
 
 
 
 172.17.0.2 
 38550 
 192.168.56.2 
 3128 
 
 
 172.17.0.2 
 49726 
 192.168.56.150 
 8888 
 
 
 172.17.0.2 
 38492 
 23.216.74.12 
 443 
 
 
 172.17.0.2 
 38508 
 104.86.237.27 
 443 
 
 
 172.17.0.2 
 38522 
 104.17.123.99 
 443 
 
 
 172.17.0.2 
 38524 
 140.82.113.4 
 443 
 
 
 172.17.0.2 
 38538 
 65.8.66.113 
 443 
 
 
 
 Block Executed in 5 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-file* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 splunkd 
 process 
 1796 
 1796 
 2507 
 2625 
 2134 
 379 
 5108 
 1400 
 762 
 46 
 762 
 3452 
 762 
 3452 
 3452 
 3452 
 
 
 splunkd_activities 
 process 
 1624 
 1624 
 3366 
 3602 
 2700 
 388 
 8136 
 1948 
 782 
 70 
 782 
 4880 
 782 
 4880 
 4880 
 4880 
 
 
 traffic 
 network-traffic 
 7 
 9 
 3366 
 3602 
 2700 
 381 
 9760 
 1948 
 782 
 70 
 782 
 4880 
 782 
 4880 
 4880 
 4880 
 
 
 *Number of related records cached.

In [2]:
# what are the attributes of `traffic` before we apply the analytics
INFO traffic

# https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/domainnamelookup
APPLY python://domain-enrichment ON traffic

# what are the attributes of `traffic` after we apply the analytics
INFO traffic

Entity Type,network-traffic
Number of Entities,7
Number of Records,9
Entity Attributes,"dst_port, dst_byte_count, dst_packets, src_port, src_byte_count, src_packets, protocols, id"
Indirect Attributes,[]
Customized Attributes,
Birth Command,find
Associated Datasource,stixshifter://bh22-linux-192.168.56.91
Dependent Variables,splunkd_activities
Entity Type,network-traffic
Number of Entities,7


In [3]:
DISP traffic ATTR src_ref.value, src_port, dst_ref.value, dst_port, x_domain_name, x_domain_organization

src_ref.value,src_port,dst_ref.value,dst_port,x_domain_name,x_domain_organization
172.17.0.2,38550,192.168.56.2,3128,,
172.17.0.2,49726,192.168.56.150,8888,,
172.17.0.2,38492,23.216.74.12,443,a23-216-74-12.deploy.static.akamaitechnologies.com,"Akamai Technologies, Inc. (AKAMAI)"
172.17.0.2,38508,104.86.237.27,443,a104-86-237-27.deploy.static.akamaitechnologies.com,"Akamai Technologies, Inc. (AKAMAI)"
172.17.0.2,38522,104.17.123.99,443,,"Cloudflare, Inc. (CLOUD14)"
172.17.0.2,38524,140.82.113.4,443,lb-140-82-113-4-iad.github.com,"GitHub, Inc. (GITHU)"
172.17.0.2,38538,65.8.66.113,443,server-65-8-66-113.yvr50.r.cloudfront.net,"Amazon.com, Inc. (AMAZO-4)"


In [5]:
# https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/piniponmap
APPLY python://pin-IP-on-map ON traffic

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<head> 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 
 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>
 
 <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>
 <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-1.12.4.min.js"></script>
 <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css"/>
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_9b32ebba9bd6b08dccf579db59a15593 {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 </style>
 
 <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet-minimap/3.6.1/Control.MiniMap.js"></script>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/leaflet-minimap/3.6.1/Control.MiniMap.css"/>
</head>
<body> 
 
 <div class="folium-map" id="map_9b32ebba9bd6b08dccf579db59a15593" ></div>
 
</body>
<script> 
 
 var map_9b32ebba9bd6b08dccf579db59a15593 = L.map(
 "map_9b32ebba9bd6b08dccf579db59a15593",
 {
 center: [37.751, -97.822],
 crs: L.CRS.EPSG3857,
 zoom: 10,
 zoomControl: true,
 preferCanvas: false,
 }
 );

 

 
 
 var tile_layer_192236b7cc1d521215a14f09c9eb8218 = L.tileLayer(
 "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
 {"attribution": "Data by \u0026copy; \u003ca href=\"http://openstreetmap.org\"\u003eOpenStreetMap\u003c/a\u003e, under \u003ca href=\"http://www.openstreetmap.org/copyright\"\u003eODbL\u003c/a\u003e.", "detectRetina": false, "maxNativeZoom": 18, "maxZoom": 18, "minZoom": 0, "noWrap": false, "opacity": 1, "subdomains": "abc", "tms": false}
 ).addTo(map_9b32ebba9bd6b08dccf579db59a15593);
 
 
 var marker_8de4843b647328506d93647587019e04 = L.marker(
 [33.844, -84.4784],
 {}
 ).addTo(map_9b32ebba9bd6b08dccf579db59a15593);
 
 
 marker_8de4843b647328506d93647587019e04.bindTooltip(
 `<div>
 23.216.74.12
 </div>`,
 {"sticky": true}
 );
 
 
 var marker_f8f29b60f6bb6c5d58704cbd9d451f42 = L.marker(
 [37.751, -97.822],
 {}
 ).addTo(map_9b32ebba9bd6b08dccf579db59a15593);
 
 
 marker_f8f29b60f6bb6c5d58704cbd9d451f42.bindTooltip(
 `<div>
 104.86.237.27
 </div>`,
 {"sticky": true}
 );
 
 
 var marker_5efbe7819f62dc866aa9b15e7ed25c43 = L.marker(
 [37.751, -97.822],
 {}
 ).addTo(map_9b32ebba9bd6b08dccf579db59a15593);
 
 
 marker_5efbe7819f62dc866aa9b15e7ed25c43.bindTooltip(
 `<div>
 140.82.113.4
 </div>`,
 {"sticky": true}
 );
 
 
 var marker_6751cc2c30a25ee69da1bedfd2844c93 = L.marker(
 [37.751, -97.822],
 {}
 ).addTo(map_9b32ebba9bd6b08dccf579db59a15593);
 
 
 marker_6751cc2c30a25ee69da1bedfd2844c93.bindTooltip(
 `<div>
 65.8.66.113
 </div>`,
 {"sticky": true}
 );
 
 
 var tile_layer_09110040bfcca42f6cd8ba39631e5e22 = L.tileLayer(
 "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
 {"attribution": "Data by \u0026copy; \u003ca href=\"http://openstreetmap.org\"\u003eOpenStreetMap\u003c/a\u003e, under \u003ca href=\"http://

## Beaconing Behavior of C2 Traffic

We said we will check the statistics of the C2 traffic at the end of the second huntbook, and we may find some clues of beaconing behavior. Let's try:

In [6]:
c2_proc = splunkd WHERE command_line LIKE '%192.168.56.150%'

c2_traffic = FIND network-traffic CREATED BY c2_proc

c2_traffic = TIMESTAMPED(c2_traffic)

DISP c2_traffic ATTR first_observed, src_ref.value, src_port, dst_ref.value, dst_port

first_observed,src_ref.value,src_port,dst_ref.value,dst_port
2022-07-27T14:38:30.442905703Z,224.0.0.251,5353,192.168.56.91,5353
2022-07-27T14:38:30.443064336Z,172.17.0.2,5353,172.17.0.1,5353
2022-07-27T14:38:31.443896894Z,172.17.0.2,5353,172.17.0.1,5353
2022-07-27T14:38:31.445104677Z,172.17.0.2,49738,192.168.56.150,8888
2022-07-27T14:38:31.446651846Z,172.17.0.2,49738,192.168.56.150,8888
2022-07-27T14:39:31.517374587Z,172.17.0.2,49738,192.168.56.150,8888
2022-07-27T14:39:31.521548431Z,172.17.0.2,49738,192.168.56.150,8888
2022-07-27T14:40:01.028527Z,172.17.0.2,49738,192.168.56.150,8888
2022-07-27T14:40:07.573279244Z,172.17.0.2,49738,192.168.56.150,8888
2022-07-27T14:40:31.004563Z,172.17.0.2,49738,192.168.56.150,8888


In [9]:
# let's visualize the data to make the traffic pattern explicit to human eye
# of course we can plug in a beaconing analytics here to let machine detect as well

# https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/attributeplot
APPLY python://attribute-plot ON c2_traffic WITH XPARAM=first_observed, YPARAM=id

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 
 
 
 
 
 2022-07-29T13:07:53.963858 
 image/svg+xml 
 
 
 Matplotlib v3.5.2, https://matplotlib.org/ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 15:00 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 14:40 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 14:45 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 14:50 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 14:55 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 15:05 -->
 
 
 
 
 
 
 
 
 
 
 <!-- timestamp -->
 
 
 
 
 <path id="DejaVuSans-6d" d="M 3328 2828 
Q 3544 3216 3844 3400 
Q 4144 3584 4550 3584 
Q 5097 3584 5394 3201 
Q 5691 2819 5691 2113 
L 5691 0 
L 5113 0 
L 5113 2094 
Q 5113 2597 4934 2840 
Q 4756 3084 4391 3084 
Q 3944 3084 3684 2787 
Q 3425 2491 3425 1978 
L 3425 0 
L 2847 0 
L 2847 2094 
Q 2847 2600 2669 2842 
Q 2491 3084 2119 3084 
Q 1678 3084 1418 2786 
Q 1159 2488 1159 1978 
L 1159 0 
L 581 0 
L 581 3500 
L 1159 3500 
L 1159 2956 
Q 1356 3278 1631 3431 
Q 1906 3584 2284 3584 
Q 2666 3584 2933 3390 
Q 3200 3197 3328 2828 
z
" transform="scale(0.015625)"/>
 
 <path id="DejaVuSans-73" d="M 2834 3397 
L 2834 2853 
Q 2591 2978 2328 3040 
Q 2066 3103 1784 3103 
Q 1356 3103 1142 2972 
Q 928 2841 928 2578 
Q 928 2378 1081 2264 
Q 1234 2150 1697 2047 
L 1894 2003 
Q 2506 1872 2764 1633 
Q 3022 1394 3022 966 
Q 3022 478 2636 193 
Q 2250 -91 1575 -91 
Q 1294 -91 989 -36 
Q 684 19 347 128 
L 347 722 
Q 666 556 975 473 
Q 1284 391 1588 391 
Q 1994 391 2212 530 
Q 2431 669 2431 922 
Q 2431 1156 2273 1281 
Q 2116 1406 1581 1522 
L 1381 1569 
Q 847 1681 609 1914 
Q 372 2147 372 2553 
Q 372 3047 722 3315 
Q 1072 3584 1716 3584 
Q 2034 3584 2315 3537 
Q 2597 3491 2834 3397 
z
" transform="scale(0.015625)"/>
 <path id="DejaVuSans-61" d="M 2194 1759 
Q 1497 1759 1228 1600 
Q 959 1441 959 1056 
Q 959 750 1161 570 
Q 1363 391 1709 391 
Q 2188 391 2477 730 
Q 2766 1069 2766 1631 
L 2766 1759 
L 2194 1759 
z
M 3341 1997 
L 3341 0 
L 2766 0 
L 2766 531 
Q 2569 213 2275 61 
Q 1981 -91 1556 -91 
Q 1019 -91 701 211 
Q 384 513 384 1019 
Q 384 1609 779 1909 
Q 1175 2209 1959 2209 
L 2766 2209 
L 2766 2266 
Q 2766 2663 2505 2880 
Q 2244 3097 1772 3097 
Q 1472 3097 1187 3025 
Q 903 2953 641 2809 
L 641 3341 
Q 956 3463 1253 3523 
Q 1550 3584 1831 3584 
Q 2591 3584 2966 3190 
Q 3341 2797 3341 1997 
z
" transform="scale(0.015625)"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 2 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 4 -->
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 6 -->
 
 
 <path id="DejaVuSans-36" d="M 2113 2584 
Q 1688 2584 1439 2293 
Q 1191 2003 1191 1497 
Q 1191 994 1439 701 
Q 1688 409 2113 409 
Q 2538 409 2786 701 
Q 3034 994 3034 1497 
Q 3034 2003 2786 2293 
Q 2538 2584 2113 2584 
z
M 3366 4563 
L 3366 3988 
Q 3128 4100 2886 4159 
Q 2644 4219 2406 4219 
Q 1781 4219 1451 3797 
Q 1122 3375 1075 2522 
Q 1259 2794 1537 2939 
Q 1816 3084 2150 3084 
Q 2853 3084 3261 2657 
Q 3669 2231 3669 1497 
Q 3669 778 3244 343 
Q 2819 -91 2113 -91 
Q 1303 -91 875 529 
Q 447 1150 447 2328 
Q 447 3434 972 4092 
Q 1497 4750 2381 4750 
Q 2619 4750 2861 4703 
Q 3103 4656 3366 4563 
z
" transform="scale(0.015625)"/>
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 8 -->
 
 
 <path id="DejaVuSans-38" d="M 2034 2216 
Q 1584 2216 1326 1975 
Q 1069 1734 1069 1313 
Q 1069 891 1326 650 
Q 1584 409 2034 409 
Q 2484 409 2743 651 
Q 3003 894 3003 1313 
Q 3003 1734 2745 1975 
Q 2488 2216 2034 2216 
z
M 1403 2484 
Q 997 2584 770 2862 
Q 544 3141 544 3541 
Q 544 4100 942 4425 
Q 1341 4750 2034 4750 
Q 2731 4750 3128 4425 
Q 3525 4100 3525 3541 
Q 3525 3141 3298 2862 
Q 3072 2584 2669 2484 
Q 3125 2378 3379 2068 
Q 3634 1759 3634 1313 
Q 3634 634 3220 271 
Q 2806 -91 2034 -91 
Q 1263 -91 848 271 
Q 434 634 434 1313 
Q 434 1759 690 2068 
Q 947 2378 1403 2484 
z
M 1172 3481 
Q 1172 3119 1398 2916 
Q 1625 2713 2034 2713 
Q 2441 2713 2670 2916 
Q 290

## Analytics != End of Hunt-Flow

Kestrel analytics is just a hunt step, which can be followed by any other hunt step to compose a large huntflow.

In [12]:
# e.g.: new attributes gained by an analytics could help construct filters to derive new Kestrel variables

github_traffic = traffic WHERE x_domain_organization LIKE '%GitHub%'

DISP traffic ATTR src_ref.value, src_port, dst_ref.value, dst_port, x_domain_name, x_domain_organization
DISP github_traffic ATTR src_ref.value, src_port, dst_ref.value, dst_port, x_domain_name, x_domain_organization

src_ref.value 
 src_port 
 dst_ref.value 
 dst_port 
 x_domain_name 
 x_domain_organization 
 
 
 
 
 172.17.0.2 
 38550 
 192.168.56.2 
 3128 
 
 
 
 
 172.17.0.2 
 49726 
 192.168.56.150 
 8888 
 
 
 
 
 172.17.0.2 
 38492 
 23.216.74.12 
 443 
 a23-216-74-12.deploy.static.akamaitechnologies.com 
 Akamai Technologies, Inc. (AKAMAI) 
 
 
 172.17.0.2 
 38508 
 104.86.237.27 
 443 
 a104-86-237-27.deploy.static.akamaitechnologies.com 
 Akamai Technologies, Inc. (AKAMAI) 
 
 
 172.17.0.2 
 38522 
 104.17.123.99 
 443 
 
 Cloudflare, Inc. (CLOUD14) 
 
 
 172.17.0.2 
 38524 
 140.82.113.4 
 443 
 lb-140-82-113-4-iad.github.com 
 GitHub, Inc. (GITHU) 
 
 
 172.17.0.2 
 38538 
 65.8.66.113 
 443 
 server-65-8-66-113.yvr50.r.cloudfront.net 
 Amazon.com, Inc. (AMAZO-4) 
 
 
 
 
 
 
 src_ref.value 
 src_port 
 dst_ref.value 
 dst_port 
 x_domain_name 
 x_domain_organization 
 
 
 
 
 172.17.0.2 
 38524 
 140.82.113.4 
 443 
 lb-140-82-113-4-iad.github.com 
 GitHub, Inc. (GITHU) 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-file* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 github_traffic 
 network-traffic 
 1 
 1 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 *Number of related records cached.

## Apply Analytics Executed Remotely

Invoke a Kestrel analytics via:
- [Kestrel Python analytics interface](https://kestrel.readthedocs.io/en/stable/source/kestrel_analytics_python.interface.html) (used in this demo)
- [Kestrel Docker analytics interface](https://kestrel.readthedocs.io/en/stable/source/kestrel_analytics_docker.interface.html)
- Kestrel AWS lambda analytics interface (planned)
- Kestrel msticpy analytics interface (planned)

<img src="https://kestrel.readthedocs.io/en/stable/_images/interfaces.png" alt="Kestrel interfaces" width="800" align="left"/>